## [Indexing API](https://python.langchain.com/docs/how_to/indexing/)

LangChain의 Indexing API는 **문서 소스를 벡터스토어에 효율적으로 동기화**하는 워크플로우를 제공합니다.
이를 통해 다음과 같은 이점을 얻을 수 있습니다:
- 중복된 문서가 저장되지 않도록 방지
- 변경되지 않은 문서를 다시 저장하지 않음
- 변경되지 않은 문서의 임베딩을 다시 계산하지 않음

변환된 문서 (예: 텍스트 분할 후 문서)도 추적이 가능하도록 설계되어 있습니다.

### 작동 방식

Indexing API는 `RecordManager`라는 컴포넌트를 통해 동작하며, 문서가 벡터스토어에 어떻게 기록되었는지를 추적합니다.

각 문서를 인덱싱할 때 다음 정보가 저장됩니다:

- 문서 해시 (내용 + 메타데이터 기준)
- 작성 시간
- 소스 ID (원본 문서의 출처를 나타내는 메타데이터)

이러한 정보는 `RecordManager`의 내부 데이터베이스(SQLite)에 저장됩니다.


# 🧹 Cleanup 모드

인덱싱 과정에서 기존 벡터스토어의 문서를 어떻게 정리할지 결정하는 **Cleanup 모드**를 제공합니다.  
문서 중복 방지, 삭제된 문서 제거, 병렬 처리 여부에 따라 4가지 모드를 제공합니다.

---

## Cleanup 모드 요약

| 모드           | 중복 제거 | 병렬 처리 가능 | 삭제된 문서 제거 | 수정된 문서 정리 | 정리 시점               |
|----------------|------------|------------------|------------------|------------------|--------------------------|
| `None`         | ✅         | ✅               | ❌               | ❌               | 없음 (정리 안 함)        |
| `incremental`  | ✅         | ✅               | ❌               | ✅               | 인덱싱 중 실시간         |
| `full`         | ✅         | ❌               | ✅               | ✅               | 인덱싱 종료 후 일괄 정리 |
| `scoped_full`  | ✅         | ✅               | ❌               | ✅               | 인덱싱 종료 후 정리       |

---

## 각 모드 상세 설명

### `None`
- 기존 문서를 삭제하지 않고 유지합니다.
- 새 문서는 추가되고, 기존 문서가 있으면 덮어씌워집니다.
- 삭제는 사용자가 수동으로 처리해야 합니다.

### `incremental`
- **변경된 문서만 갱신**하고, 같은 source ID를 가진 예전 문서를 실시간으로 삭제합니다.
- 중복 제거에 효과적이고 병렬 처리가 가능합니다.
- 데이터가 부분적으로 업데이트되는 워크플로우에 적합합니다.

### `full`
- **현재 로딩된 문서만 유지**하고 나머지는 모두 삭제합니다.
- 전체 데이터셋을 인덱싱할 수 있을 때 적합합니다.
- 인덱싱 중 중복이 발생할 수 있으며, 병렬 처리에는 부적합합니다.

### `scoped_full`
- `full`과 유사하지만, **같은 source ID를 가진 문서만 삭제**합니다.
- 메모리에 source ID만 유지하므로 병렬 처리도 가능합니다.
- 전체 데이터가 한 번에 로딩되지 않는 환경에서 유용합니다.

---

## 정리 시점 (Cleanup Timing)

| 모드           | 정리 시점             |
|----------------|------------------------|
| `None`         | 없음 (정리 안 함)       |
| `incremental`  | 인덱싱 중 실시간 정리   |
| `full`         | 인덱싱 종료 후 정리     |
| `scoped_full`  | 인덱싱 종료 후 정리     |

---

## ⚠️ 사용 시 주의사항

- **외부에서 직접 벡터스토어에 문서를 추가**한 경우, `incremental`, `full`, `scoped_full` 모드 사용 시 주의가 필요합니다.  
  RecordManager가 해당 문서를 인식하지 못해 실수로 삭제될 수 있습니다.

- `full`, `scoped_full` 모드 사용 시, **데이터 로더가 전체 문서를 반환해야 안전**합니다.  
  일부 문서만 반환하면, 반환되지 않은 문서가 삭제될 수 있습니다.

- `scoped_full`은 **병렬 처리와 데이터 정리에 균형**이 필요한 경우에 적합한 선택지입니다.

---


## 실습

본 튜토리얼은 로컬에 Milvus 서버가 실행되고 있다고 가정하며, 아닌 경우 루트 디렉토리의 docker-compose.yml을 실행하여 Milvus 서버를 실행합니다.

```bash
docker compose up -d
```

Milvus WebUI는 해당 주소에서 실행할 수 있습니다.

```bash
http://<HOST>:9091/webui # HOST 변경 필요
```

### 먼저 관련 패키지들을 임포트합니다.

In [ ]:
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
from langchain_milvus import Milvus
from langchain_ollama import OllamaEmbeddings
from pymilvus import connections, db

### Milvus VectorStore 초기화

- `pymilvus`의 `connections`와 `db`를 사용하여 DB를 생성 및 연결합니다.
- `langchain_milvus`의 `Milvus`를 사용하여 Milvus 벡터스토어를 초기화합니다.

In [ ]:
uri = "http://localhost:19530"
db_name = "indexing_db3"
collection_name = "test_collection3"
embeddings = OllamaEmbeddings(model="bge-m3:latest")

# DB 생성
connections.connect(uri=uri)
db.create_database(db_name)

# Milvus VectorStore 생성
vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": uri, "db_name": db_name},
    consistency_level="Strong", # 일관성(정확성) 최우선: Strong, 성능과 일관성 균형: Session, 성능 최우선: Eventually
    drop_old=True,
)

### RecordManager 초기화 및 스키마 생성

SQLite를 사용하여 레코드 정보를 저장합니다. (PostgreSQL, MySQL 등을 사용할 수 있습니다.)

In [ ]:
# RecordManager 생성 (vectorstore와 collection_name을 모두 고려하는 namespace 사용 권장)
namespace = f"milvus/{collection_name}"
db_url = "sqlite:///langchain_indexing3.sql"
record_manager = SQLRecordManager(namespace, db_url=db_url)
record_manager.create_schema()

In [ ]:
# from pymilvus import utility, Collection
# # DB 삭제
# if db_name in db.list_database():
#     db.using_database(db_name)

#     collections = utility.list_collections()
#     for collection in collections:
#         collection = Collection(name=collection)
#         collection.drop()
#         print(f"Dropped collection: {collection.name}")

#     db.drop_database(db_name)


### Helper function 정의

반복적인 테스트를 위해 레코드 매니저를 초기화하는 helper 함수를 정의합시다.

이는 `full` 모드는 새로운 리스트(문서들)로 내용을 완전히 바꾸는 점을 이용합니다.
즉, 빈 리스트를 전달하여 모든 문서들을 레코더에서 삭제합니다.

In [ ]:
def _clear():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

### 문서 인덱싱

아래와 같이 두 개의 문서를 이용해서 각 모드의 특징을 확인해봅시다.

In [ ]:
# Indexing
doc1 = Document(page_content="kitty", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="doggy", metadata={"source": "doggy.txt"})

### None 모드

동일한 문서 다섯 개를 None 모드로 인덱싱해봅시다.

아래의 셀을 실행하면 하나의 문서만 추가된 것을 확인할 수 있습니다.

```python
{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
```


In [ ]:
index(
    [doc1, doc1, doc1, doc1, doc1],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source"
)

이번에는 doc1과 doc2를 None 모드로 인덱싱해봅시다.

아래의 셀을 실행하면 기존에 추가된 doc1은 skip되고 doc2는 추가된 것을 확인할 수 있습니다.

```python
{'num_added': 1, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 0}
```

In [ ]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

문서들을 모두 지우고 다시 한 번 doc1과 doc2를 None 모드로 인덱싱해봅시다.

이번에는 doc1과 doc2가 모두 추가된 것을 확인할 수 있습니다.

```python
{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
```

In [ ]:
_clear()

In [ ]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

In [ ]:
vectorstore.similarity_search("cat", k=5)

### incremental 모드

모든 문서들을 지우고, 이번에는 `incremental` 모드를 이용해 동일한 문서 두 개를 인덱싱해봅시다. 

`None` 모드와 마찬가지로 초기 인덱싱에서는 하나의 문서만 추가됩니다.

```python
{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
```

In [ ]:
_clear()

In [ ]:
index(
    [doc1, doc1],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
vectorstore.col.flush()
vectorstore.col.num_entities

이번에는 doc2를 추가해봅시다.

```python
{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
```

In [ ]:
index(
    [doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

In [ ]:
vectorstore.col.flush()
vectorstore.col.num_entities

In [ ]:
vectorstore.similarity_search("")

## 문서 변경

이번에는 문서가 변경되었을 때 어떻게 처리하는지 확인해보겠습니다.

우선 두 개의 문서를 준비하고
- kitty.txt: kitty1 kitty2 kitty3 kitty4 kitty5
- doggy.txt: doggy1 doggy2 doggy3

이 문서들을 쪼개어 인덱싱하면 8개의 문서가 추가됩니다.

```python
{'num_added': 8, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}
```

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

doc1 = Document(page_content="kitty1 kitty2 kitty3 kitty4 kitty5", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="doggy1 doggy2 doggy3", metadata={"source": "doggy.txt"})

new_docs = CharacterTextSplitter(
    separator=" ", keep_separator=False, chunk_size=6, chunk_overlap=0
).split_documents([doc1, doc2])
new_docs

In [ ]:
_clear()

In [ ]:
index(
    new_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
vectorstore.similarity_search("cat", k=10)

"kitty.txt"의 내용을 변경하였을 때, 어떻게 추가되는지 확인해봅시다.

우선 문서의 내용을 아래와 같이 변경합니다:
"kitty1 kitty2 kitty3 kitty4 **kitty5**" -> "**kitty5** kitty6 kitty7 kitty8 kitty9"

이를 splitter를 통해 5개의 청크로 분할합니다. 

이 청크들을 indexing하면, 변경이 없는 kitty5는 skip되고, kitty6~kitty9는 추가됩니다.
```python
{'num_added': 4, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 4}
```


In [ ]:
changed_kitty_docs = [
    Document(page_content="kitty5 kitty6 kitty7 kitty8 kitty9", metadata={"source": "kitty.txt"}),
]
new_changed_docs = CharacterTextSplitter(
    separator=" ", keep_separator=False, chunk_size=6, chunk_overlap=0
).split_documents(changed_kitty_docs)
new_changed_docs

In [ ]:
index(
    new_changed_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)


In [ ]:
len(vectorstore.similarity_search("cat", k=10))

## Appendix

### index 함수 매개변수
| 이름              | 설명|
|------------------|------------------|
docs_source        | 문서를 제공하는 로더 또는 문서 리스트|
record_manager     | 어떤 문서가 새로 추가/수정되었는지 추적하는 관리 객체|
vector_store       | 인덱싱 대상 벡터 저장소 (예: Milvus, FAISS 등)|
batch_size         | 한 번에 인덱싱할 문서 수|
cleanup            | 불필요한 문서를 어떻게 정리할지 선택 (full, incremental, scoped_full, None)|
source_id_key      | 문서의 출처(source id)를 구분하는 키|
cleanup_batch_size | 정리할 때 몇 개씩 삭제할지 정하는 크기|
force_update       | 기존 문서가 있어도 무조건 업데이트할지 여부|
upsert_kwargs      | 벡터 저장소의 add_documents 또는 upsert 메서드에 추가 인자 전달 가능|
